In [ ]:
import utils
import pandas as pd
import os
import utils

import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from patchify import patchify, unpatchify
from tensorflow.keras.models import load_model
from ipywidgets import interact

from matplotlib.lines import Line2D



In [ ]:
masks = glob.glob("./data/final_images/train/masks/*.png")
for mask in masks:
    img = cv2.imread(mask, 0)
    print(np.unique(img))
    binary_mask = np.where(img == 1, 1, 0)
    #cv2.imwrite(mask, binary_mask)

In [ ]:
masks = glob.glob("./data/final_images/validation/masks/*.png")
for mask in masks:
    img = cv2.imread(mask, 0)
    print(np.unique(img))
    binary_mask = np.where(img == 1, 1, 0)
    #cv2.imwrite(mask, binary_mask)

In [ ]:
masks = glob.glob("./data/final_images/test/masks/*.png")
for mask in masks:
    img = cv2.imread(mask, 0)
    print(np.unique(img))
    binary_mask = np.where(img == 1, 1, 0)
    #cv2.imwrite(mask, binary_mask)

# Dont touch

In [ ]:
import utils
import utils._file_operations

import os
import glob
import cv2
import re
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import keras
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.utils import to_categorical
from keras.metrics import MeanIoU

import keras.backend as K
import tensorflow as tf

def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha*((1-p)^gamma)*log(p)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediction value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -y_true*K.log(y_pred)
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = alpha * y_true * K.pow((1-y_pred), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f*y_pred_f)
    return (2.0 * intersection + 1.0)/(K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f*y_pred_f)
    return (intersection + 1.0)/(K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

def dice_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * tf.keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

# Code

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda


def unet_model(n_classes=1, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=3):

    inputs = Input((None, None, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

        alpha_t = y_true * alpha + (tf.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (tf.ones_like(y_true) - y_true) * (tf.ones_like(y_true) - y_pred) + tf.keras.backend.epsilon()
        fl = - alpha_t * tf.keras.backend.pow((tf.ones_like(y_true) - p_t), gamma) * tf.keras.backend.log(p_t)
        return tf.keras.backend.mean(fl)
    return focal_loss_fixed

def prepare_image_set(directory):
    """
    Given a directory preprocess the images and returns an X, y set
    """
    #Capture training image info as a list
    train_images = []

    for img_path in glob.glob(os.path.join(directory, "images", "*.png")):
        img = cv2.imread(img_path, cv2.COLOR_BGR2RGB)       
        train_images.append(img)
        
    #Convert list to array for machine learning processing        
    train_images = np.array(train_images)

    #Capture mask/label info as a list
    train_masks = [] 

    for mask_path in glob.glob(os.path.join(directory, "masks", "*.png")):
        mask = cv2.imread(mask_path, 0)       
        train_masks.append(mask)
       

    train_masks = np.array(train_masks)
    # Normalize images
    train_images = train_images/255.0

    return train_images, train_masks


def train_model(TRAIN_CONFIG):

    utils._file_operations.create_folder(os.path.join("./trained_models", TRAIN_CONFIG["results_folder"]))

    
    X_train, y_train = prepare_image_set(TRAIN_CONFIG["training_path"])
    X_val, y_val = prepare_image_set(TRAIN_CONFIG["validation_path"])
    X_test, y_test = prepare_image_set(TRAIN_CONFIG["test_path"])

    print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background 

    # Convert masks to categorical (should have shape (height, width, n_classes))
    y_train_cat = np.float32(y_train)
    y_val_cat = np.float32(y_val)
    y_test_cat = np.float32(y_test)

    # Check that the shape of all images is the same
    assert X_train.shape[1] == X_val.shape[1] == X_test.shape[1]


    IMG_HEIGHT = X_train.shape[1]
    IMG_WIDTH  = X_train.shape[2]
    IMG_CHANNELS = X_train.shape[3]

    def get_model(network):
        return unet_model(n_classes=TRAIN_CONFIG["n_classes"], IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)
    
    keras.utils.set_random_seed(32)
    model = get_model(network = "unet")
    
    optimizer = keras.optimizers.Adam(learning_rate=TRAIN_CONFIG["learning_rate"])
    callback_ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=TRAIN_CONFIG["early_stopping_patience"], restore_best_weights=True)
    callback_LR = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=TRAIN_CONFIG["lr_mod_factor"], patience=TRAIN_CONFIG["lr_on_plateau_patience"])

    model.compile(optimizer=optimizer, loss=focal_loss(), metrics=['accuracy', jacard_coef])
    #model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy', jacard_coef])

    #model.summary()

    #If starting with pre-trained weights. 
    #model.load_weights('???.hdf5')

    start = datetime.now()
    history = model.fit(X_train, y_train_cat, 
                        batch_size = TRAIN_CONFIG["batch_size"], 
                        verbose=TRAIN_CONFIG["verbose"], 
                        epochs=TRAIN_CONFIG["epochs"], 
                        validation_data=(X_val, y_val_cat), 
                        callbacks=[callback_ES, callback_LR],
                        # sample_weight=class_weights,
                        shuffle=True)
    stop = datetime.now()
    execution_time = stop - start
    print(f"The Neural Network training took {execution_time}")


    # Model evaluation
    _, acc, jaq = model.evaluate(X_test, y_test_cat, batch_size=8)
    print("Accuracy is = ", (acc * 100.0), "%")


    y_pred = model.predict(X_test, batch_size=6)
    y_pred_argmax = np.argmax(y_pred, axis=3)
    # y_pred_argmax_unstacked = utils.unstack_labels(y_pred_argmax)
    # y_test_unstacked = utils.unstack_labels(y_test)


    conf_matrix = confusion_matrix(y_test.reshape(-1), y_pred_argmax.reshape(-1))
    clf_report = classification_report(y_test.reshape(-1), y_pred_argmax.reshape(-1), output_dict=True)


    print(classification_report(y_test.reshape(-1), y_pred_argmax.reshape(-1)))


    IOU_keras = MeanIoU(num_classes=TRAIN_CONFIG["n_classes"])  
    IOU_keras.update_state(y_test, y_pred_argmax)
    mean_IoU = IOU_keras.result().numpy()
    print("Mean IoU =", mean_IoU)



# Test code

In [ ]:
networks = ["unet", "deeplabv3", "attention_unet", "attention_resunet", "unet_pluplus"]

execution_time = pd.Timestamp.now()
TRAIN_CONFIG = {
    "training_path":"./data/final_images/train",
    "validation_path":"./data/final_images/validation", 
    "test_path":"./data/final_images/test",
    "network":"unet",
    "save_model":True,
    "downscaling_factor":32, #If specified you can keep track of performance in the excel file
    "n_classes":1,
    "batch_size": 16,
    "epochs": 100,
    "validation_split":0.1,
    "test_split":0.1,
    "learning_rate":0.001,
    "early_stopping_patience":30,
    "lr_on_plateau_patience":10,
    "lr_mod_factor":0.1,
    "execution_time": execution_time.strftime("%Y%m%d_%H%M%S"),
    "results_folder": "results_" + execution_time.strftime("%Y%m%d_%H%M%S"),
    "verbose": 1 if os.name=="nt" else 0 # dissables verbose if we run on Snellius
}

train_model(TRAIN_CONFIG)